In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import h5py
import math
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Conv2D, AveragePooling2D, MaxPooling2D, BatchNormalization, Flatten, ZeroPadding2D, Activation
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc 
from matplotlib.pyplot import imshow
%matplotlib inline
import keras.backend as K
K.set_image_data_format("channels_last")
K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
pip install git+git://github.com/wau/keras-rl2.git --upgrade --no-deps

  Cloning git://github.com/wau/keras-rl2.git to /tmp/pip-req-build-mcnygsul
  Running command git clone -q git://github.com/wau/keras-rl2.git /tmp/pip-req-build-mcnygsul
  Created wheel for keras-rl2: filename=keras_rl2-1.0.4-cp36-none-any.whl size=53199 sha256=23e33a96409ad395c508b3c240f12ae70e73d8ae10703a36a6a6070d1f4b01fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-pm2q5cg1/wheels/2b/a2/35/5c5d276e34f74ea063a6f2498e9b4a059027fa9da74c7fd6a3
Successfully built keras-rl2


In [4]:
from google.colab import files
uploaded = files.upload()

Saving test_signs.h5 to test_signs.h5
Saving train_signs.h5 to train_signs.h5


In [14]:

def load_dataset():
    train_dataset = h5py.File('train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels

    test_dataset = h5py.File('test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) #  test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) #  test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

## Implementing the Identity block function


In [6]:
def identityBlock(X, f, filters, stage, block):
  # Naming Conventions
  conv_name_base = "res" + str(stage) + block + "_branch"
  bn_name_base = "bn" + str(stage) + block + "_branch"

  F1, F2, F3 = filters
  X_shortcut = X

  # First Block
  X = Conv2D(F1, kernel_size = (1,1), strides = (1,1), padding = "valid", name = conv_name_base + "2a", kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
  X = Activation("relu")(X)

  # Second Block
  X = Conv2D(F2, kernel_size = (f,f), strides = (1,1), padding = "same", name = conv_name_base + "2b", kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
  X = Activation("relu")(X)

  # Third Block
  X = Conv2D(F3, kernel_size = (1,1), strides = (1,1), padding = "valid", name = conv_name_base + "2c", kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)

  X = Add()([X, X_shortcut])
  X = Activation("relu")(X)

  return X

## Implementing the Convolutional Block function

In [7]:
def convolutionalBlock(X, f, filters, stage, block,s = 2):
  # Naming Convention
  conv_name_base = "res" + str(stage) + block + "_branch"
  bn_name_base = "bn" + str(stage) + block + "_branch"

  F1, F2, F3 = filters
  X_shortcut = X

  # First Block
  X = Conv2D(F1, kernel_size = (1,1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
  X = Activation("relu")(X)


  # Second Block
  X = Conv2D(F2, kernel_size= (f,f), strides = (1,1), padding = "same", name = conv_name_base + "2b", kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
  X = Activation("relu")(X)
  
  # Third Block
  X = Conv2D(F3, kernel_size = (1,1), strides = (1,1), padding = "valid", name = conv_name_base + "2c", kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)

  # Shortcut Path
  X_shortcut = Conv2D(F3, kernel_size = (1,1), strides = (s,s), padding = "same", name = conv_name_base + "1", kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + "1")(X_shortcut)

  X = Add()([X, X_shortcut])
  X = Activation("relu")(X)
  
  return X

## Building the ResNet50 architechture

In [8]:
def ResNet50(input_shape = (64,64,3), classes = 6):
  X_input = Input(input_shape)
  X = ZeroPadding2D((3,3))(X_input)

  # Stage 1
  X = Conv2D(64, (7,7), strides = (2,2), name = "conv1", kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = "bn1")(X)
  X = Activation("relu")(X)
  X = MaxPooling2D((3,3), strides = (2,2))(X)

  # Stage 2
  X = convolutionalBlock(X, f = 3, filters = [64,64,256], stage = 2, block = "a", s = 1)
  X = identityBlock(X, f = 3, filters = [64,64,256], stage = 2, block = "b")
  X = identityBlock(X, f = 3, filters = [64,64,256], stage = 2, block = "c")

  # Stage 3
  X = convolutionalBlock(X, f = 3, filters = [128,128,512], stage = 3, block = "a", s = 2)
  X = identityBlock(X, f = 3, filters = [128,128,512], stage = 3, block = "b")
  X = identityBlock(X, f = 3, filters = [128,128,512], stage = 3, block = "c")
  X = identityBlock(X, f = 3, filters = [128,128,512], stage = 3, block = "d")

  # Stage 4
  X = convolutionalBlock(X, f = 3, filters = [256,256,1024], stage = 4, block = "a", s = 2)
  X = identityBlock(X, f = 3, filters = [256,256,1024], stage = 4, block = "b")
  X = identityBlock(X, f = 3, filters = [256,256,1024], stage = 4, block = "c")
  X = identityBlock(X, f = 3, filters = [256,256,1024], stage = 4, block = "d")
  X = identityBlock(X, f = 3, filters = [256,256,1024], stage = 4, block = "e")
  X = identityBlock(X, f = 3, filters = [256,256,1024], stage = 4, block = "f")

  # Stage 5
  X = convolutionalBlock(X, f =3, filters = [512,512,2048], stage = 5, block = "a", s = 2)
  X = identityBlock(X, f = 3, filters = [512,512,2048], stage = 5, block = "b")
  X = identityBlock(X, f = 3, filters = [512,512,2048], stage = 5, block = "c")

  # Average Pooling
  X = AveragePooling2D((2,2), name = "avgPool")(X)

  X = Flatten()(X)
  X = Dense(classes, activation = "softmax", name = "fc" + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
  model = Model(inputs = X_input, outputs = X, name = "ResNet50")

  return model

In [9]:
model = ResNet50(input_shape=(64,64,3), classes = 6)
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [10]:
# Loading the SIGNS dataset 
X_train, y_train, X_test, y_test, classes = load_dataset()

X_train = X_train / 255
X_test = X_test / 255
y_train = convert_to_one_hot(y_train, 6).T
y_test = convert_to_one_hot(y_test, 6).T
print(f"Number of training examples : {X_train.shape[0]}")
print(f"Number of testing examples : {X_test.shape[0]}")
print(f"X_train shape : {X_train.shape}")
print(f"X_test shape : {X_test.shape}")
print(f"y_train shape : {y_train.shape}")
print(f"y_test shape : {y_test.shape}")

Number of training examples : 1080
Number of testing examples : 120
X_train shape : (1080, 64, 64, 3)
X_test shape : (120, 64, 64, 3)
y_train shape : (1080, 6)
y_test shape : (120, 6)


In [11]:
model.fit(X_train, y_train, epochs = 30, batch_size = 32)

Epoch 1/30
1080/1080 [==============================] - 21s 20ms/step - loss: 1.7118 - accuracy: 0.4778
Epoch 2/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.7499 - accuracy: 0.7602
Epoch 3/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.3247 - accuracy: 0.8954
Epoch 4/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.1774 - accuracy: 0.9444
Epoch 5/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.2474 - accuracy: 0.9194
Epoch 6/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.3414 - accuracy: 0.9000
Epoch 7/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.1676 - accuracy: 0.9593
Epoch 8/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.1606 - accuracy: 0.9481
Epoch 9/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.1246 - accuracy: 0.9639
Epoch 10/30
1080/1080 [==============================] - 3s 3ms/step - loss: 0.1

In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss : {loss}")
print(f"Model accuracy is : {accuracy*100}%")

120/120 [==============================] - 1s 5ms/step
Loss : 0.3482056260108948
Model accuracy is : 94.16666626930237%


In [13]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________